In [1]:
import ee
import xee
import xarray as xr
import xarray


In [2]:
# ee.Initialize()
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')


In [18]:
col_id='ECMWF/ERA5_LAND/HOURLY'

year = str(2020)
prefix = "WuHan_Met_"
fout = prefix + col_id.replace("/", "_") + "_" + str(year) + ".nc"
print(fout)


import os.path
os.path.isfile(fout)


WuHan_Met_ECMWF_ERA5_LAND_HOURLY_2020.nc


False

In [23]:
import os.path


def ee_col_download_year(region, col_id='ECMWF/ERA5_LAND/HOURLY', 
                         year=2022, save=False, prefix=""):

    fout = prefix + col_id.replace("/", "_") + "_" + str(year) + ".nc"
    if os.path.isfile(fout): return
    
    ic = (ee.ImageCollection(col_id)
          # .filterDate('2000-01-01', '2023-01-01')
          .filter(ee.Filter.calendarRange(year, year, 'year'))
          .select(['temperature_2m', 'u_component_of_wind_10m', 'v_component_of_wind_10m'])
    )
    print(fout)
    print(ic.size().getInfo())
    
    ds = xarray.open_dataset(
        ic,
        engine='ee',
        projection=ic.first().select(0).projection(),
        geometry=region
    )
    if save: ds.to_netcdf(fout);
    ds

def ee_col_download_years(region, col_id='ECMWF/ERA5_LAND/HOURLY', 
    year_beg=2022, year_end=2022, save=False, prefix=""):
    
    for year in range(year_beg, year_end + 1): 
        try: 
            ee_col_download_year(region, col_id, year, save, prefix)
        except Exception as e:
            print(str(e))


In [24]:
%%time

# range = c(113.5, 115, 29.75, 31.5)
region = ee.Geometry.Rectangle(113.5, 29.75, 115, 31.5)

prefix = "WuHan_Met_"
# ee_col_download_year(region, year=2000, save=True)
ee_col_download_years(region, year_beg=2023, year_end=2023, save=True)


False ECMWF_ERA5_LAND_HOURLY_2023.nc
6619


c:\ProgramData\Miniconda3\envs\rgee\lib\site-packages\xee\ext.py:514: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  xarray.Variable(self.primary_dim_name, primary_coord),


CPU times: total: 12 s
Wall time: 1min 3s
